In [4]:
from inference_sdk import InferenceHTTPClient

client = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="49V2UYgr1auzNYjpi8kQ"
)

result = client.run_workflow(
    workspace_name="autovrs-fdupl",
    workflow_id="find-hats-gloves-and-masks",
    images={
        "image": r"C:\Users\sonng\Code\DemoCCTVAI\first_frame.jpg"
    },
    use_cache=True # cache workflow definition for 15 minutes
)

In [5]:
print(result)

[{'predictions': {'image': {'width': 2560, 'height': 1920}, 'predictions': [{'width': 412.0, 'height': 246.0, 'x': 1330.0, 'y': 270.0, 'confidence': 0.97265625, 'class_id': 0, 'class': 'hat', 'detection_id': '03b92012-e55f-4a0f-865c-e2d81737853b', 'parent_id': 'image', 'rle_mask': {'size': [1920, 2560], 'counts': 'QjkQ2>]k19F:H6G8^Ob0D=C<H8J6M3L3N3L3N3L4M3L4M2N2N3N1N3M2N2N3M2N2O1N2M4M2O1N2N2O1N2O1O1N2O1O1N2O1O1O1O1O1O1N2O1O1O100O10000O100O1O100O010O1O100O100O10000O10000O100O1000000O01000O10000O100O0100000O10000O10000O1000000O100O10O010000O10000O010O1000000O0100000000O1001N101O0000001O00001O1O001O001O00001O001O001O00001O0010O0001O00001O001O001O0001O01O0000001O000010O0001O001O0010O0001O00001O01O01O00010O000010O0001O010O000010O0000010O00010O000010O0001O0010O0001O0010O01O0010O0001O0010O01O001O001O1O001O10O01O1O001O1O1O1O1O1O1O1O1O1O1O1O1O1O1O2N1O1O1O1O1O2N1O2N1O2N2N1O2N1O2N2N2N2N1O1O2M2O2N2N2N2M3N2N2N1O2M2O2N1N3N1O1N2O1O1O2M2O001N2O1O001N101O0O2O000O2O0O101N101N101N101N10001N1O2N100O2O0O2O

In [1]:
!pip install inference-sdk

  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---- ----------------------------------- 4.5/38.8 MB 22.3 MB/s eta 0:00:02
   ----------- ---------------------------- 11.5/38.8 MB 28.8 MB/s eta 0:00:01
   ------------------- -------------------- 18.9/38.8 MB 32.2 MB/s eta 0:00:01
   ------------------------ --------------- 24.1/38.8 MB 31.1 MB/s eta 0:00:01
   -------------------------------- ------- 31.5/38.8 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------  38.8/38.8 MB 31.6 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 30.5 MB/s  0:00:01
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 36.0 MB/s  0:00:00
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)

  Attempting uninstall: pillow

    Found existing installation: pillow 12.0.0

    Uninstalling pillow-12.0.0:
